In [ ]:
import pymongo
import pandas as pd
import os
import pickle
import time

import gaode

client = pymongo.MongoClient(host="localhost", port=27017)
db = client.zufang6
collection = db.lianjia

def count_estate():
    #获取杭州市所有小区的名字，通过集合去重
    """
    out : 返回一个包含所有小区名字的集合
    """
    if not os.path.exists("estate.txt"):
        print("\n读取mongo数据库数据。。。")
        total_docs = collection.find({},{"_id":0, "estate":1, "district":1})
        total_estate = set()
        for x in total_docs:
            try:
                estate = x.get("estate")
                district = x.get("district")
                total_estate.update({district+estate})
            except:
                print("wrong...")
        print(len(total_estate))
        print(total_estate)
        with open("estate.txt", "wb") as f:
            pickle.dump(total_estate, f)
    else:
        print("\n读取文本文件...")
        with open("estate.txt", "rb") as f:
            total_estate = pickle.load(f)
            print(len(total_estate))

    return total_estate


def request_gaode_all_bus(start, total_estate, is_random_proxies=True):
    #访问、计算所有小区到该小区的时间。
    # 需要用到“高德.py”那个脚本
    """
    function: 使用“高德”脚本进行请求，担心循环请求中出问题，加了一层try。（如果该轮出错，则该小区的bus时间就得不到了，略过。。。）
    out: 包含所有小区到该小区的时间的字典
    """
    bus_time_dic = {}
    count = 1
    for estate in total_estate:
        # try:
        end = estate
        d1 = gaode.crawl_1(start, is_random_proxies)
        d2 = gaode.crawl_1(end, is_random_proxies)
        # 睡一下，防止被封ip。（但是我全程同步请求，其实相当于睡好久了。。。。）
        # time.sleep(0.2)
        bus_dic = gaode.crawl_2(d1, d2, is_random_proxies)
        my_bus_dic = gaode.parse(bus_dic)
        print("\n\n")
        print(my_bus_dic)
        print("\n\n")
        bus_time_dic.update({estate:my_bus_dic["total_time"]})
        # print(bus_time_dic)
        print("           ================完成第 {0} 条数据===================".format(count))
        print(" ===========================================================================")
        print("\n\n\n\n")
        # except Exception as e:
        #     pri   nt("error:------------", e)
        # break 放在try内好像就失效了。。。现在拿出来了
        # if count == 2:
        #     break
        count += 1
    print(bus_time_dic)
    return bus_time_dic

def save_to_csv(start, bus_time_dic):
    bus_time_dic = pd.Series(bus_time_dic)
    bus_time_pd = pd.DataFrame(bus_time_dic, columns=["total_time"])
    bus_time_pd.to_csv("{}.csv".format(start), index=True, encoding="gb18030")



def main():
    # try:
    total_estate = count_estate()
    start_name = "下城区武林广场"
    bus_time_dic = request_gaode_all_bus(start_name, total_estate, is_random_proxies=False)
    save_to_csv(start_name, bus_time_dic)
    # except Exception as e:
    #     print("曹尼玛。。。。。。。。。。：", e)

if __name__ == "__main__":
    main()



读取文本文件...
2423
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=下城区武林广场
{'name': '武林广场', 'district': '浙江省杭州市下城区', 'x': '120.163277', 'y': '30.270999', 'id': 'B023B02841'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=江干区广宇上东名筑
{'name': '广宇·上东名筑', 'district': '浙江省杭州市江干区', 'x': '120.225002', 'y': '30.360432', 'id': 'B0FFF09PPQ'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_2) 
https://www.amap.com/service/nav/bus?night=1&group=1&pure_walk=1&date=2018-12-6&time=15-30&type=0&eta=1&x1=120.163277&y1=30.270999&poiid1=B023B02841&ad1=330103&x2=120.225002&y2=30.360432&poiid2=B0FFF09PPQ&ad2=330104



{'total_time': 58, 'total_length': 15693, 'total_foot_length': 1390, 'middle_lst': [{'bus_name': '535路(武林广场--天都城)', 'start_name': '武林广场', 'end_name': '沿山', 'foot_time': 4, 'foot_length': 275, 'bus_wait_time': 12, 'bus_drive_time': 26, 'driver_length': 14303, 'this_time': 4

200
it's ok: ------ (crawl_2) 
https://www.amap.com/service/nav/bus?night=1&group=1&pure_walk=1&date=2018-12-6&time=15-30&type=0&eta=1&x1=120.163277&y1=30.270999&poiid1=B023B02841&ad1=330103&x2=120.236984&y2=30.164295&poiid2=B023B0A4NC&ad2=330104



{'total_time': 38, 'total_length': 15959, 'total_foot_length': 623, 'middle_lst': [{'bus_name': '地铁1号线(临平--湘湖)', 'start_name': '武林广场', 'end_name': '湘湖', 'foot_time': 5, 'foot_length': 331, 'bus_wait_time': 5, 'bus_drive_time': 24, 'driver_length': 15336, 'this_time': 34, 'this_length': 15667, 'this_foot_length': 331}], 'end_foot_time': 4, 'end_foot_length': 292}



           ================完成第 7 条数据===================





获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=下城区武林广场
{'name': '武林广场', 'district': '浙江省杭州市下城区', 'x': '120.163277', 'y': '30.270999', 'id': 'B023B02841'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&t

200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=余杭区金都西花庭
{'name': '金都·西花庭', 'district': '浙江省杭州市余杭区', 'x': '119.951302', 'y': '30.27428', 'id': 'B023B14FVA'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_2) 
https://www.amap.com/service/nav/bus?night=1&group=1&pure_walk=1&date=2018-12-6&time=15-30&type=0&eta=1&x1=120.163277&y1=30.270999&poiid1=B023B02841&ad1=330103&x2=119.951302&y2=30.27428&poiid2=B023B14FVA&ad2=330104



{'total_time': 85, 'total_length': 26648, 'total_foot_length': 1763, 'middle_lst': [{'bus_name': '506路(武林门北--余杭)', 'start_name': '武林门北', 'end_name': '余杭二院', 'foot_time': 9, 'foot_length': 611, 'bus_wait_time': 12, 'bus_drive_time': 48, 'driver_length': 24885, 'this_time': 69, 'this_length': 25496, 'this_foot_length': 611}], 'end_foot_time': 16, 'end_foot_length': 1152}



           ================完成第 14 条数据===================





获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city

200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=下城区武林广场
{'name': '武林广场', 'district': '浙江省杭州市下城区', 'x': '120.163277', 'y': '30.270999', 'id': 'B023B02841'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_1) 
https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=江干区万科中央公园
{'name': '万科·中央公园', 'district': '浙江省杭州市江干区', 'x': '120.228262', 'y': '30.295153', 'id': 'B0FFGNTNR8'}
获取随机头部
获取随机曲奇
200
it's ok: ------ (crawl_2) 
https://www.amap.com/service/nav/bus?night=1&group=1&pure_walk=1&date=2018-12-6&time=15-30&type=0&eta=1&x1=120.163277&y1=30.270999&poiid1=B023B02841&ad1=330103&x2=120.228262&y2=30.295153&poiid2=B0FFGNTNR8&ad2=330104



{'total_time': 41, 'total_length': 8472, 'total_foot_length': 1365, 'middle_lst': [{'bus_name': '地铁1号线(湘湖--临平)', 'start_name': '武林广场', 'end_name': '彭埠', 'foot_time': 5, 'foot_length': 331, 'bus_wait_time': 9, 'bus_drive_time': 12, 'driver_length': 7107, 'this_time': 26, 'this_length': 7438, 'this_foot_

获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (

获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (crawl_1): sleeping...........
 https://www.amap.com/service/poiTipslite?&city=330100&type=dir&words=上城区清江路114-2号
获取随机头部
获取随机曲奇
200

it's wrong for requesting (